In [ ]:
pip install transformers torch rouge-score

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [ ]:
pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 45.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import csv

# Load the translated dataset
file_path = '/content/drive/MyDrive/shop-chatbot/clothesshop.vi.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Preprocess the text
def preprocess_text(text):
    return text.lower().strip()

data['Context'] = data['Context'].apply(preprocess_text)
data['Question'] = data['Question'].apply(preprocess_text)
data['Answer'] = data['Answer'].apply(preprocess_text)

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        context = self.data.iloc[index]['Context']
        question = self.data.iloc[index]['Question']
        answer = self.data.iloc[index]['Answer']
        input_text = "context: " + context + " question: " + question
        inputs = self.tokenizer(input_text, max_length=self.max_len, truncation=True, padding='max_length', return_tensors="pt")
        labels = self.tokenizer(answer, max_length=self.max_len, truncation=True, padding='max_length', return_tensors="pt")
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = labels['input_ids'].squeeze()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "context": context,
            "question": question,
            "answer": answer
        }

In [ ]:
# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("VietAI/vit5-base")
model = T5ForConditionalGeneration.from_pretrained("VietAI/vit5-base")

# Prepare the dataset
train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/shop-chatbot/bert_results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()



special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,3.242600,0.099872
1000,0.074300,0.089774
1500,0.042000,0.098103
2000,0.024200,0.105889


TrainOutput(global_step=2370, training_loss=0.7162599702424641, metrics={'train_runtime': 3005.8879, 'train_samples_per_second': 3.154, 'train_steps_per_second': 0.788, 'total_flos': 5772920802508800.0, 'train_loss': 0.7162599702424641, 'epoch': 10.0})

In [ ]:
# Load ROUGE metric
rouge = load_metric("rouge")

# Function to generate predictions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def generate_predictions(dataset, model, tokenizer, device, max_len=512):
    model.eval()
    model.to(device)
    predictions = []
    actuals = []
    contexts = []
    questions = []

    for i in range(len(dataset)):
        data = dataset[i]
        input_text = "context: " + data['context'] + " question: " + data['question']
        inputs = tokenizer(input_text, max_length=max_len, truncation=True, padding='max_length', return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_len, num_beams=4, early_stopping=True)

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        actual = data['answer']

        predictions.append(pred)
        actuals.append(actual)
        contexts.append(data['context'])
        questions.append(data['question'])

    return contexts, questions, predictions, actuals

# Generate predictions on the test dataset
contexts, questions, predictions, actuals = generate_predictions(test_dataset, model, tokenizer, device)

# Calculate ROUGE scores
results = rouge.compute(predictions=predictions, references=actuals)
print("ROUGE-1: ", results["rouge1"])
print("ROUGE-2: ", results["rouge2"])
print("ROUGE-L: ", results["rougeL"])

# Save the predictions and actual answers to a CSV file
output_file = '/content/drive/MyDrive/shop-chatbot/predictions.csv'
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Context', 'Question', 'Predicted Answer', 'Actual Answer'])
    for context, question, pred, actual in zip(contexts, questions, predictions, actuals):
        writer.writerow([context, question, pred, actual])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ROUGE-1:  AggregateScore(low=Score(precision=0.7085665636082827, recall=0.6984357689325202, fmeasure=0.6951256442305851), mid=Score(precision=0.7310413798471733, recall=0.719847132983056, fmeasure=0.7166034614495767), high=Score(precision=0.7542555274905641, recall=0.7437630763175409, fmeasure=0.7398899983603892))
ROUGE-2:  AggregateScore(low=Score(precision=0.47456734872547696, recall=0.4632960127830937, fmeasure=0.4639055028224244), mid=Score(precision=0.513073864968963, recall=0.5026692408700293, fmeasure=0.5030017375064251), high=Score(precision=0.5506984060797847, recall=0.5403091207217204, fmeasure=0.540575314139959))
ROUGE-L:  AggregateScore(low=Score(precision=0.5611535981104274, recall=0.5501320426567047, fmeasure=0.5493526006391783), mid=Score(precision=0.5960257354636621, recall=0.5847413960165664, fmeasure=0.5840667057391449), high=Score(precision=0.6290484403497726, recall=0.6153540791445583, fmeasure=0.6162405474623671))
